In [21]:
import os 
import torch
import torchmetrics
import torch.nn.functional as F

In [22]:
from torch import nn 
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

from PIL import Image


In [23]:
import os
from PIL import Image

class Dataset():
    def __init__(self, filepath, filelist, transforms=None):
        self.filelist = filelist
        self.filepath = filepath
        self.transform = transforms

    def __len__(self):
        return len(self.filelist)
    
    def __getitem(self, index):
        imgpath = os.path.join(self.filepath, self.filelist[index])
        img = Image.open(imgpath)

        if "dog" in imgpath:
            label = 1
        else:
            label = 0

        if self.transform is not None:
            img = self.transform(img)
        
        return img, label

# Example usage
train_dir = "dogs-vs-cats/train."
test_dir = "dogs-vs-cats/test1"


In [24]:
import torch.utils
import torch.utils.data


train_files = os.listdir(train_dir)
test_files = os.listdir(test_dir)

transformations = transforms.Compose([transforms.Resize((60,60), transforms.ToTensor())])



In [25]:
train = Dataset(train_files, train_dir, transformations)
val = Dataset(test_files, test_dir, transformations)



ValueError: Sum of input lengths does not equal the length of the input dataset!

In [ ]:
import torch
from torch.utils.data import random_split

# Assuming 'train' is your dataset
dataset_length = len(train)  # Get the length of the dataset
split_ratio = [0.8, 0.2]  # Example split ratios

split_lengths = [int(r * dataset_length) for r in split_ratio]

# Adjust the last split to make sure the sum is equal to the dataset length
split_lengths[-1] = dataset_length - sum(split_lengths[:-1])

train_split, val_split = random_split(train, split_lengths)


In [ ]:
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Assuming you have your data and labels as tensors
train_data = torch.randn(100, 3, 32, 32)  # Example training data
train_labels = torch.randint(0, 2, (100,))  # Example training labels
val_data = torch.randn(20, 3, 32, 32)  # Example validation data
val_labels = torch.randint(0, 2, (20,))  # Example validation labels

train_dataset = MyDataset(train_data, train_labels)
val_dataset = MyDataset(val_data, val_labels)


In [ ]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class LitDataModule(pl.LightningDataModule):
    def __init__(self, train_dataset, val_dataset, batch_size=32):
        super().__init__()
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.batch_size = batch_size

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False)

class LitModel(pl.LightningModule):
    def __init__(self):
        super(LitModel, self).__init__()

        self.conv1 = nn.Sequential(nn.Conv2d(3, 16, 3), nn.ReLU(), nn.MaxPool2d(2, 2))
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, 3), nn.ReLU(), nn.MaxPool2d(2, 2))
        self.conv3 = nn.Sequential(nn.Conv2d(32, 64, 3), nn.ReLU(), nn.MaxPool2d(2, 2))

        self.fc1 = nn.Sequential(nn.Flatten(), nn.Linear(64 * 5 * 5, 256), nn.Linear(256, 128), nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(128, 2))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return F.softmax(x, dim=1)

    def training_step(self, batch, batch_idx):
        data, label = batch
        output = self(data)
        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(output, label)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        data, label = batch
        output = self(data)
        loss_fn = nn.CrossEntropyLoss()
        val_loss = loss_fn(output, label)
        self.log('val_loss', val_loss)
        return val_loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.02)

# Assuming train and val are your training and validation datasets
train_dataset = MyDataset(train_data, train_labels)
val_dataset = MyDataset(val_data, val_labels)

# Create instances of the data module and model
data_module = LitDataModule(train_dataset, val_dataset, batch_size=32)
model = LitModel()

# Create a PyTorch Lightning trainer
trainer = pl.Trainer()

# Train the model
trainer.fit(model, data_module)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
d:\image\Lib\site-packages\pytorch_lightning\loops\utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | conv1 | Sequential | 448    | train
1 | conv2 | Sequential | 4.6 K  | train
2 | conv3 | Sequential | 18.5 K | train
3 | fc1   | Sequential | 442 K  | train
4 | fc2   | Sequential | 258    | train
---------------------------------------------
466 K     Trainable params
0         Non-trainable params
466 K     Total params
1.866     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (20x256 and 1600x256)

In [ ]:
# Assuming train and val are your training and validation datasets
train_dataset = train
val_dataset = val

# Create instances of the data module and model
data_module = LitDataModule(train_dataset, val_dataset, batch_size=32)
model = LitModel()

# Create a PyTorch Lightning trainer with the auto scaling batch size plugin
trainer = pl.Trainer(plugins=[pl.plugins.AutoScaleBatchSizePlugin()])

# Run batch size finder
trainer.tune(model, datamodule=data_module)

# Train the model
trainer.fit(model, data_module)


AttributeError: module 'pytorch_lightning.plugins' has no attribute 'AutoScaleBatchSizePlugin'

In [ ]:
# Assuming train and val are your training and validation datasets
train_dataset = train
val_dataset = val

# Create instances of the data module and model with an initial batch size
initial_batch_size = 32
data_module = LitDataModule(train_dataset, val_dataset, batch_size=initial_batch_size)
model = LitModel()

# Create a PyTorch Lightning trainer
trainer = pl.Trainer()

# Train the model
trainer.fit(model, data_module)

# If you encounter memory issues, reduce the batch size and try again
# data_module.batch_size = 16
# trainer.fit(model, data_module)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | conv1 | Sequential | 448    | train
1 | conv2 | Sequential | 4.6 K  | train
2 | conv3 | Sequential | 18.5 K | train
3 | fc1   | Sequential | 442 K  | train
4 | fc2   | Sequential | 258    | train
---------------------------------------------
466 K     Trainable params
0         Non-trainable params
466 K     Total params
1.866     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

TypeError: 'Dataset' object is not subscriptable